# Sentiment Analysis Report
In this report, I will explain the methodology I followed to solve the sentiment analysis problem.

The idea is to write the reusable code for the pre-processing and the post the training part that can be used with different models.


## 1. Preprocessing Step: 
The training and the validation data had non-relevant columns such as UMID, Entity etc. While loading the data in Pandas dataframe, the would occupy additional space in the memory. Also, the text had a lot of HTML tags, Punctuation, Numbers and Stopwords. I created the two new CSV files one for each train and the validate dataset. 

In [2]:
import pandas as pd
import os
local_path = "/Users/ayush/ml-team-competition"
remote_path = "/home/q-ayushpandey/ml-team-competition/"
os.chdir(local_path)
xl = pd.ExcelFile("validate.xlsx")
# xl = pd.ExcelFile("ml-team-competition/train.xlsx")
#xl.sheet_names
validate = xl.parse("Human")
validate_text_sentiment = validate[['Message','Human', 'Prediction']]
from bs4 import BeautifulSoup 
import re
import nltk
#nltk.download()  # Download text data sets, including stop words
from nltk.corpus import stopwords # Import the stop word list 
stops = set(stopwords.words("english"))   

def text_to_words(raw_text):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_text).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    # stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    clear_text = " ".join(meaningful_words)
    validate_text_sentiment["Message"][i] = clear_text 
    #print i
    #return( " ".join( meaningful_words ))  

num_text = validate_text_sentiment["Message"].size #701989
# num_text = 100000
for i in xrange(0, num_text):
    # Call our function for each one, and add the result to the list of
    # clean Text
    text_to_words(validate_text_sentiment["Message"][i])

validate_text_sentiment.to_csv("validate.csv")

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## 2. Training

I tried two different models: 
1. A **Random Forest Model**
2. A Bigram Suport Vector Machine Classifier with bagging

### 2.1 Training Random Forest Model
This step can further be divided into 2 sub-steps:

1. **Creating feature vector from the text data:** Used Bag of Words technique provided by scikit-learn. The Bag of Words model learns a vocabulary from all of the documents, then models each document by counting the number of times each word appears.


2. **Hyper-parameter Tuning:** The Random Forest Model has hperparameters such as number of trees, height of the tree. I tuned the model using 10-fold cross validation on number of tree for values 100, 150, 200, 250 using the selection criteria as the default provided by the scikit-learn package.

**Note 1:** *scikit-learn allows us to run our K-fold validation in parallel utilizing the cores in our system.*

**Note 2:** *I tried tuning on F1 score but ran into some error. We need to look into it in future.*

The model would write the predicted value and the actual value to **rf.csv** file.


In [ ]:
import pandas as pd
import os
local_path = "/Users/ayush/ml-team-competition"
remote_path = "/home/q-ayushpandey/ml-team-competition/"
os.chdir(local_path)

train = pd.read_csv("train.csv",index_col='Id')
train.head
num_text = train["Text"].size
print num_text
# Initialize an empty list to hold the clean Text
# clean_train_text = []

# # Loop over each Text; create an index i that goes from 0 to the length
# # of the movie review list 
# for i in xrange(0, num_text):
#     # Call our function for each one, and add the result to the list of
#     # clean Text
#     if train["Text"][i]==None:
#         train["Text"][i]=""
#         print "None String"
    
#     clean_train_text.append(train["Text"][i])
#     print i

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 100) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(train["Text"].values.astype('U'))
# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()
# Take a look at the words in the vocabulary
# vocab = vectorizer.get_feature_names()
# print vocab
print "train_data_features prepared"
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier()

# 10-fold Cross-Validation using Grid Search 
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
scoring = ['precision_macro', 'recall_macro', 'f1_macro']
# param_grid = {'n_estimators': [100, 150, 200, 250, 300]}
param_grid = {'n_estimators': [1, 2, 3, 4]}
from sklearn.grid_search import GridSearchCV
grid_clf = GridSearchCV(forest, param_grid, cv=10, scoring='f1_macro', n_jobs=-1)
#scores = cross_validate(clf, iris.data, iris.target, scoring=scoring, cv=5, return_train_score=False)
grid_clf.fit(train_data_features, train["Sentiment"])

print "Cross-Validation done"
best_model = grid_clf.best_estimator_
print grid_clf.best_params_
print grid_clf.grid_scores_

# Initialize a Random Forest classifier with 100 trees
#forest = RandomForestClassifier(n_estimators = 10) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
#forest = forest.fit(train_data_features,train["Sentiment"] )

# Predict using the existing model
validate = pd.read_csv("validate.csv",index_col='Id')
clean_validate_text = []
validate_num_text = validate["Message"].size

# Loop over each Text; create an index i that goes from 0 to the length
# of the movie review list 
# for i in xrange(0, validate_num_text):
#     # Call our function for each one, and add the result to the list of
#     # clean Text
#     clean_validate_text.append(validate["Message"][i])
#     print i

#from sklearn.feature_extraction.text import CountVectorizer
test_data_features = vectorizer.transform(validate["Message"].values.astype('U'))
test_data_features = test_data_features.toarray()
result = best_model.predict(test_data_features)
actual_prediction = pd.DataFrame( data={"Prediction":result, "Actual":validate["Human"]} )
actual_prediction.to_csv("rf.csv")
print "Result written to DataFrame"

# from sklearn.metrics import f1_score, precision_score, recall_score
# f1_score(actual_prediction["Actual"], actual_prediction["Prediction"], average=None)
# precision_score(actual_prediction["Actual"], actual_prediction["Prediction"], average=None)
# recall_score(actual_prediction["Actual"], actual_prediction["Prediction"], average=None)


### 2.2 SVM Bigram Model

Inspired by the [this](https://www.linkedin.com/pulse/best-ai-algorithms-sentiment-analysis-muktabh-mayank/) blogpost, I tried Support Vector Machine with Linear Kernel and Bigram Countevectorizer.

**Note:** The training was very slow so instead I tried ensembles of 10 SVCs that are trained on subsets of the data to reduce the number of records per classifier which in turn reduced the training time complexity.

In [ ]:
import pandas as pd
import os
local_path = "/Users/ayush/ml-team-competition"
remote_path = "/home/q-ayushpandey/ml-team-competition/"
os.chdir(local_path)

train = pd.read_csv("train.csv",index_col='Id')
train.head
num_text = train["Text"].size
print num_text
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#vectorizer = TfidfVectorizer(min_df=2,max_df = 0.95,sublinear_tf=True,use_idf=True)
vectorizer = CountVectorizer(analyzer='char_wb', tokenizer = None, preprocessor = None, stop_words = None, max_features = 10, ngram_range=(2, 2))
train_vectors = vectorizer.fit_transform(train["Text"].values.astype('U'))

validate = pd.read_csv("validate.csv",index_col='Id')
validate_num_text = validate["Message"].size
test_vectors = vectorizer.transform(validate["Message"].values.astype('U'))

# Model
from sklearn import svm
from sklearn.ensemble import BaggingClassifier
# svm_rbf = svm.SVC(C = 1, gamma = 0.001, kernel='linear')
n_estimators = 10
svm_rbf = BaggingClassifier(svm.SVC(C = 1, gamma = 0.001, kernel='linear'),max_samples=1.0 / n_estimators, n_estimators=n_estimators)
model = svm_rbf.fit(train_vectors,train["Sentiment"])

result = model.predict(test_vectors)


actual_prediction = pd.DataFrame( data={"Prediction":result, "Actual":validate["Human"]} )
actual_prediction.to_csv("svm_ngram_bagging.csv")
print "Result written to DataFrame"

## 3. Prediction and Accuracy

Once we have the **model_name.csv** file, we can use the **sklearn.metrics** functionality to calculate the F1 score, recall and the precision and write the result to **model_name_score.txt** file.

In [ ]:
import pandas as pd
import os
local_path = "/Users/ayush/ml-team-competition"
remote_path = "/home/q-ayushpandey/ml-team-competition/"
os.chdir(local_path)

actual_prediction = pd.read_csv("rf.csv",index_col='Id')
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
calculated_f1_score = f1_score(actual_prediction["Actual"], actual_prediction["Prediction"], labels=["Neutral","Positive","Negative"],average=None)
print calculated_f1_score
calculated_precision_score = precision_score(actual_prediction["Actual"], actual_prediction["Prediction"], average=None)
print calculated_precision_score
calculated_recall_score = recall_score(actual_prediction["Actual"], actual_prediction["Prediction"], average=None)
print calculated_recall_score

file = open("rf_score.txt","w") 
 
file.write("f1_score\n")
file.write("%s\n" % calculated_f1_score)
file.write("precision_score\n")
file.write("%s\n" % calculated_precision_score)
file.write("recall_score\n")
file.write("%s\n" % calculated_recall_score)

target_names = ['Nagative', 'Neutral', 'Positive']
classification_report = classification_report(actual_prediction["Actual"], actual_prediction["Prediction"], target_names=target_names)
print classification_report
file.write("%s\n" % classification_report)

file.close()

## 4. Results

### 4.1 Random Forest Model

In [ ]:
precision    recall  f1-score   support

   Nagative       0.61      0.39      0.48       269
    Neutral       0.55      0.77      0.64       427
   Positive       0.72      0.53      0.61       304

avg / total       0.62      0.59      0.59      1000

### 4.2 SVM Bigram Model

In [ ]:
precision    recall  f1-score   support

   Nagative       0.25      0.64      0.36       269
    Neutral       0.39      0.28      0.33       427
   Positive       0.38      0.01      0.02       304

avg / total       0.35      0.30      0.24      1000